## Setup

```python
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)
```


In [ ]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)


In [1]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader


In [2]:
# this is required since Juypyter notebook is not async by default.
import nest_asyncio
nest_asyncio.apply()

## Load data

In [2]:
docs="/teamspace/studios/this_studio/retrieval-augmented-generation/data/"
loader = DirectoryLoader(docs, glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

## Split document into nodes

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [5]:
len(texts)

83

In [4]:
texts[3]

Document(page_content="that's that's clearly happened um there's also economies of economies of scale and building simpler simpler\ndevices that do one thing incredibly well um especially with consumer devices\nthough it is it's consumer consumer devices in particular are very taste driven do I think that we're gonna you\nknow go back to a world of you know giant phones with lots of buttons I don't um\nI think that this this this technological move into kind of simpler\nlooking devices aesthetic devices is probably permanent\nso is this framework part of testing the MVP because what I'm struggling with this\nespecially for Hardware products how to use these framework because initially\nyou are going to start with something over time it well well not necessarily I mean so\nyou're a hardware person is that is that why you're asking tell me tell me about what you're working on I'd like to\nlike like one thing but\nbut are embedded in Furniture yeah oh awesome right but then\nthe technolog

In [7]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [8]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [9]:
retriever = vectordb.as_retriever()

In [10]:
# docs = retriever.get_relevant_documents("What is Product Company Gap?")
docs = retriever.invoke("What is Product Company Gap?")


In [11]:
len(docs)

4

In [12]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [13]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [15]:
# full example
query = "What is Product Company Gap?"
llm_response = qa_chain.invoke(query)
process_llm_response(llm_response)

 Product Company Gap is a term used to describe the challenge that companies face in transitioning from having a successful product to becoming a successful and sustainable company. This involves considerations such as go-to-market strategies, pricing, and creating a business model that allows for scalability.


Sources:
/teamspace/studios/this_studio/retrieval-augmented-generation/data/part1.txt
/teamspace/studios/this_studio/retrieval-augmented-generation/data/part1.txt
/teamspace/studios/this_studio/retrieval-augmented-generation/data/part1.txt
/teamspace/studios/this_studio/retrieval-augmented-generation/data/part2.txt


In [ ]:
# To cleanup, you can delete the collection

# vectordb.delete_collection()
# vectordb.persist()
# delete the directory
# !rm -rf db/